In [1]:
import torch
print("GPU available : -> " , torch.cuda.is_available())
print("GPU version : -> ", torch.cuda.get_device_name() if torch.cuda.is_available() else " not found")

GPU available : ->  True
GPU version : ->  Tesla T4


In [2]:
from transformers import pipeline
import pandas as pd
import time

In [3]:
translator = pipeline("translation" ,model="Helsinki-NLP/opus-mt-en-ur" , device = 0)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/816k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/848k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [4]:
def translate_to_spanish(text):
    """Translate English text to Spanish"""
    try:
        if pd.isna(text) or text.strip() == "": # here we are cleaning data remove null value remove space
            return ""

        result = translator(str(text)) # text convert into the string or imp thing translator just accept string and other type may be crash
         # inside translator -> tokenizer -> string , convert into  numbers [2, 7592, 2088, 3]
         # NN calculation -> GPU main
         # genrate spanish tokenizer
         # detokenizer -> number convert into the spanish string
        return result[0]['translation_text']
    except Exception as e:
        print(f"Error translating: {text[:50]}... -> {e}")
        return f"[Translation Error]"

In [5]:
df = pd.read_csv("sarcasm_filtered.csv")
print("data load .....", df)

data load .....                                                   text    binary multiclass
0    it'd be nice if missguided actually had stock ...      Hope    Sarcasm
1    Find me a mass shooting in Utah where conceale...  Not Hope    Sarcasm
2    Oh, I LOVE semantics games Well, viability now...  Not Hope    Sarcasm
3    So your answer to the question then is "Nothin...  Not Hope    Sarcasm
4    Clearly you don't give a damn, because if you ...  Not Hope    Sarcasm
..                                                 ...       ...        ...
687  And if the state decides to keep abortion lega...  Not Hope    Sarcasm
688  Repent, you Christ-denying, gay bashing radica...  Not Hope    Sarcasm
689  Agreeing with the right to dunk on the centre ...  Not Hope    Sarcasm
690  i love it when christians say this...pay who, ...  Not Hope    Sarcasm
691  naming my son onickolas in honor of my love fo...      Hope    Sarcasm

[692 rows x 3 columns]


In [6]:
len(df)

692

In [7]:
csv_file = 'sarcasm_filtered.csv'
text_column = 'text'
output_file = 'output_file_urdu.csv'

# Start timer
start_time = time.time()
print(f"Reading CSV file: {csv_file}")

# Load CSV file
try:
    df = pd.read_csv(csv_file)
    print(f"Loaded successfully: {len(df)} rows")
    print(f"Column names: {list(df.columns)}")
except Exception as e:
    print(f"Error loading CSV: {e}")
    exit()

# Check if column exists
if text_column not in df.columns:
    print(f"Column '{text_column}' not found")
    exit()

# Initialize variables
urdu_translations = []
total_rows = len(df)
batch_size = 50

# Process in batches
for i in range(0, total_rows, batch_size):
    batch_end = min(i + batch_size, total_rows)
    batch_text = df[text_column].iloc[i:batch_end].tolist()

    print(f"Processing rows {i+1} to {batch_end}")

    try:
        # Translate batch
        batch_result = translator([str(text) for text in batch_text])

        # Debug: Check result format
        print("Sample result:", batch_result[0])

        batch_urdu = [result["translation_text"] for result in batch_result]
        urdu_translations.extend(batch_urdu)

    except Exception as e:
        print(f"Batch error: {e}")
        # Fallback: translate one by one
        for text in batch_text:
            if not text or text.strip() == "":
                urdu_translations.append("")
            else:
                try:
                    result = translator(str(text))
                    urdu_translations.append(result[0]['translation_text'])
                except:
                    urdu_translations.append("[Translation Error]")

    # Progress update
    progress = ((i + batch_size) / total_rows) * 100
    print(f"✅ Progress: {min(progress, 100):.1f}% complete")

# Add translations to dataframe
df["urdu_translation"] = urdu_translations

# Select columns to save
columns_to_save = ['binary', 'multiclass', 'urdu_translation']

# Check if columns exist
missing_columns = [col for col in columns_to_save if col not in df.columns]
if missing_columns:
    print(f"⚠️ Warning: Missing columns: {missing_columns}")
    columns_to_save = [col for col in columns_to_save if col in df.columns]

print(f"📝 Saving columns: {columns_to_save}")

# Create final dataframe
final_df = df[columns_to_save].copy()

# Save to CSV
final_df.to_csv(output_file, index=False)

# Summary
end_time = time.time()
duration = end_time - start_time

print(f"\n🎉 Translation Complete!")
print(f"📁 Output saved as: {output_file}")
print(f"📋 Columns saved: {list(final_df.columns)}")
print(f"⏱️ Total time: {duration:.2f} seconds")
print(f"🚀 Average speed: {len(df)/duration:.2f} translations/second")

# Show sample results
print(f"\n📋 Sample Results:")
for i in range(min(5, len(final_df))):
    print(f"{i+1}. ", end="")
    for col in final_df.columns:
        print(f"{col}: {final_df[col].iloc[i]} | ", end="")
    print()
    print("-" * 60)


Reading CSV file: sarcasm_filtered.csv
Loaded successfully: 692 rows
Column names: ['text', 'binary', 'multiclass']
Processing rows 1 to 50
Sample result: {'translation_text': 'یہ اچھا ہو جائے گا اگر یہ اصل میں صحیح طرح سے گم ہو جائے تو یہ اچھا ہو جائے گا'}
✅ Progress: 7.2% complete
Processing rows 51 to 100
Sample result: {'translation_text': 'اِس لیے آپ کو لگتا ہے کہ آپ نے ایک گُناہ کیا ہے اور آپ کے گُناہ کیا ہے ۔'}
✅ Progress: 14.5% complete
Processing rows 101 to 150
Sample result: {'translation_text': 'اس کے ممکنہ طور پر میں سمجھتا ہوں کہ خدا نے پیزا کو بنایا ہے اور اسے مخصوص مقصد کے لئے مخصوص مقصد کے ساتھ رکھا ہے اس نے اسے بہت زیادہ استعمال کِیا ہے ۔'}
✅ Progress: 21.7% complete
Processing rows 151 to 200
Sample result: {'translation_text': 'میرا خیال ہے کہ کوئی بھی اسے بیت\u200cایل بیت\u200cایل میں سے نہیں کرنا چاہتا تھا۔ (ہنسی)'}
✅ Progress: 28.9% complete
Processing rows 201 to 250
Sample result: {'translation_text': 'ایسا لگتا ہے کہ کسی کو اس حقیقت سے کوئی روک نہیں سکتا کہ وہ

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Sample result: {'translation_text': 'اس سے پہلے میں نے اپنی جہالت کا اعتراف کرنا ضروری ہے، اس سے پہلے، میں نے سوچا تھا کہ آپ کا پوسٹ ہو سکتا ہے کہ مَیں کوئی اور بھی ہو سکتا ہوں - میں نے اس کے بارے میں غلط نتیجہ کیا ہے؟'}
✅ Progress: 72.3% complete
Processing rows 501 to 550
Sample result: {'translation_text': 'مجھے پسند ہے جب یہ صحیح طور پر ہم سے پتہ چلتا ہے کہ یہ عورت کیا ہے، شاید میں یہ کہہ سکتا ہوں کہ یہ واقعی تباہ ہو گیا ہے کہ یہ واقعی کوئی تکلیف نہیں پہنچتی'}
✅ Progress: 79.5% complete
Processing rows 551 to 600
Sample result: {'translation_text': 'اِس کے بعد اُنہوں نے دیکھا کہ اُن سب کو بین\u200cالاقوامی سفر کے دوران بین\u200cالاقوامی سفر کِیا گیا ہے ۔'}
✅ Progress: 86.7% complete
Processing rows 601 to 650
Sample result: {'translation_text': 'میرا خیال ہے کہ آپ کو غلط ہو گیا ہے. میں نہیں ہوں. نہ میں کوئی کام شائع ہوا ہے اور نہ ہی غیر قانونی طور پر ترجمہ کیا گیا ہے.'}
✅ Progress: 93.9% complete
Processing rows 651 to 692
Sample result: {'translation_text': 'مَیں نے دلیری کے ساتھ 